In [1]:
import os
from datetime import date, datetime
from random import randint, choices, random
from zoneinfo import ZoneInfo

import django
# for easier visualization it is recommended to use pandas to render data...
# if pandas is not installed, you may install it with this command: pip install -U pandas
# pandas is not a dependecy of django_ledger...
import pandas as pd

# Set your django settings module if needed...
os.environ['DJANGO_SETTINGS_MODULE'] = 'dev_env.settings'

# if using jupyter notebook need to set DJANGO_ALLOW_ASYNC_UNSAFE as "true"
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

# change your working directory as needed...
os.chdir('../')

django.setup()

from django_ledger.models.entity import EntityModel
from django_ledger.models.items import ItemModel
from django_ledger.models.invoice import InvoiceModel
from django_ledger.models.bill import BillModel
from django_ledger.models.estimate import EstimateModel
from django.contrib.auth import get_user_model
from django_ledger.io import roles


# Get Your Entity Administrator UserModel

In [2]:
# change this to your preferred django username...
MY_USERNAME = 'ceo_user'
MY_PASSWORD = 'NeverUseMe|VeryInsecure!'
UserModel = get_user_model()

try:
    user_model = UserModel.objects.get(username__exact=MY_USERNAME)
except:
    user_model = UserModel(username=MY_USERNAME)
    user_model.set_password(MY_PASSWORD)
    user_model.save()

In [3]:
entity_model = EntityModel.objects.get(name__exact='My Good Looking LLC')

# Create an Entity Model

In [4]:
entity_model = EntityModel.create_entity(
    name='One Big Company, LLC',
    admin=user_model,
    use_accrual_method=True,
    fy_start_month=1
)

In [5]:
entity_model.slug

'one-big-company-llc-dx9z2zej'

# Chart of Accounts

## Create a Default Chart of Accounts

In [6]:
entity_model.has_default_coa()

False

In [7]:
default_coa_model = entity_model.create_chart_of_accounts(
    assign_as_default=True,
    commit=True,
    coa_name='My QuickStart CoA'
)

In [8]:
default_coa_model

<ChartOfAccountModel: one-big-company-llc-dx9z2zejhe83f7-coa: My QuickStart CoA>

# Populate Entity with Random Data

### Define a Start Date for Transactions

In [9]:
START_DTTM = datetime(year=2022, month=10, day=1, tzinfo=ZoneInfo('UTC'))

In [10]:
entity_model.populate_random_data(start_date=START_DTTM)

### EntityModel has now a Default Chart of Accounts

In [11]:
entity_model.has_default_coa()

True

In [12]:
default_coa_model = entity_model.get_default_coa()

# Chart of Accounts (CoA)

## Django Ledger support multiple chart of accounts.

In [13]:
another_coa_model = entity_model.create_chart_of_accounts(
    assign_as_default=False,
    commit=True,
    coa_name='My Legacy Chart of Accounts'
)

# Accounts

## Get All Accounts

In [14]:
coa_qs, coa_map = entity_model.get_all_coa_accounts()
pd.DataFrame(coa_map[default_coa_model].values())

,created,updated,path,depth,numchild,uuid,code,name,role,role_default,balance_type,locked,active,coa_model_id
0,2023-11-13 17:38:44.234131+00:00,2023-11-13 17:38:44.234138+00:00,001400010009,3,0,690baf2c-e77f-4f0e-b4fb-46eb02696410,1010,Cash,asset_ca_cash,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
1,2023-11-13 17:38:44.240975+00:00,2023-11-13 17:38:44.240981+00:00,001400010005,3,0,2f4fd01d-8194-4c78-b7ae-7dac3627f438,1050,Short Term Investments,asset_ca_mkt_sec,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
2,2023-11-13 17:38:44.248853+00:00,2023-11-13 17:38:44.248859+00:00,00140001000O,3,0,f6de4cef-abfa-416d-b115-42eab4290a4b,1100,Accounts Receivable,asset_ca_recv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
3,2023-11-13 17:38:44.251788+00:00,2023-11-13 17:38:44.251794+00:00,00140001000B,3,0,6d556011-8e89-4b27-9eea-5f7f33478238,1110,Uncollectibles,asset_ca_uncoll,True,credit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
4,2023-11-13 17:38:44.237139+00:00,2023-11-13 17:38:44.237146+00:00,001400010002,3,0,12ed5dee-51dc-4d83-8354-5434fc19da18,1200,Inventory,asset_ca_inv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2023-11-13 17:38:44.529581+00:00,2023-11-13 17:38:44.529591+00:00,001400050009,3,0,57cb12bd-45e7-470b-bf80-140445ad3764,6294,Garbage,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
84,2023-11-13 17:38:44.543776+00:00,2023-11-13 17:38:44.543788+00:00,001400050001,3,0,01218b4a-1cc6-4e81-8153-a34da5ff7f2e,6295,Electricity,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
85,2023-11-13 17:38:44.552585+00:00,2023-11-13 17:38:44.552599+00:00,00140005000K,3,0,8ab7033f-db7b-460f-a5fe-6fe5d6c22e4f,6300,Property Management,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
86,2023-11-13 17:38:44.556906+00:00,2023-11-13 17:38:44.556916+00:00,00140005000Y,3,0,e3c44191-37fd-4108-a16d-67ddc6814229,6400,Vacancy,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae


### New CoA does not have any accounts yet...

In [15]:
pd.DataFrame(coa_map[another_coa_model].values())

""


## Get Default CoA Accounts

In [16]:
default_coa_accounts_qs = entity_model.get_default_coa_accounts()
pd.DataFrame(default_coa_accounts_qs.values())

,created,updated,path,depth,numchild,uuid,code,name,role,role_default,balance_type,locked,active,coa_model_id
0,2023-11-13 17:38:44.234131+00:00,2023-11-13 17:38:44.234138+00:00,001400010009,3,0,690baf2c-e77f-4f0e-b4fb-46eb02696410,1010,Cash,asset_ca_cash,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
1,2023-11-13 17:38:44.240975+00:00,2023-11-13 17:38:44.240981+00:00,001400010005,3,0,2f4fd01d-8194-4c78-b7ae-7dac3627f438,1050,Short Term Investments,asset_ca_mkt_sec,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
2,2023-11-13 17:38:44.248853+00:00,2023-11-13 17:38:44.248859+00:00,00140001000O,3,0,f6de4cef-abfa-416d-b115-42eab4290a4b,1100,Accounts Receivable,asset_ca_recv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
3,2023-11-13 17:38:44.251788+00:00,2023-11-13 17:38:44.251794+00:00,00140001000B,3,0,6d556011-8e89-4b27-9eea-5f7f33478238,1110,Uncollectibles,asset_ca_uncoll,True,credit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
4,2023-11-13 17:38:44.237139+00:00,2023-11-13 17:38:44.237146+00:00,001400010002,3,0,12ed5dee-51dc-4d83-8354-5434fc19da18,1200,Inventory,asset_ca_inv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2023-11-13 17:38:44.529581+00:00,2023-11-13 17:38:44.529591+00:00,001400050009,3,0,57cb12bd-45e7-470b-bf80-140445ad3764,6294,Garbage,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
84,2023-11-13 17:38:44.543776+00:00,2023-11-13 17:38:44.543788+00:00,001400050001,3,0,01218b4a-1cc6-4e81-8153-a34da5ff7f2e,6295,Electricity,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
85,2023-11-13 17:38:44.552585+00:00,2023-11-13 17:38:44.552599+00:00,00140005000K,3,0,8ab7033f-db7b-460f-a5fe-6fe5d6c22e4f,6300,Property Management,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
86,2023-11-13 17:38:44.556906+00:00,2023-11-13 17:38:44.556916+00:00,00140005000Y,3,0,e3c44191-37fd-4108-a16d-67ddc6814229,6400,Vacancy,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae


## Get CoA Accounts by CoA Model

In [17]:
coa_accounts_by_coa_model_qs = entity_model.get_coa_accounts(coa_model=default_coa_model)
pd.DataFrame(coa_accounts_by_coa_model_qs.values())

,created,updated,path,depth,numchild,uuid,code,name,role,role_default,balance_type,locked,active,coa_model_id
0,2023-11-13 17:38:44.234131+00:00,2023-11-13 17:38:44.234138+00:00,001400010009,3,0,690baf2c-e77f-4f0e-b4fb-46eb02696410,1010,Cash,asset_ca_cash,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
1,2023-11-13 17:38:44.240975+00:00,2023-11-13 17:38:44.240981+00:00,001400010005,3,0,2f4fd01d-8194-4c78-b7ae-7dac3627f438,1050,Short Term Investments,asset_ca_mkt_sec,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
2,2023-11-13 17:38:44.248853+00:00,2023-11-13 17:38:44.248859+00:00,00140001000O,3,0,f6de4cef-abfa-416d-b115-42eab4290a4b,1100,Accounts Receivable,asset_ca_recv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
3,2023-11-13 17:38:44.251788+00:00,2023-11-13 17:38:44.251794+00:00,00140001000B,3,0,6d556011-8e89-4b27-9eea-5f7f33478238,1110,Uncollectibles,asset_ca_uncoll,True,credit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
4,2023-11-13 17:38:44.237139+00:00,2023-11-13 17:38:44.237146+00:00,001400010002,3,0,12ed5dee-51dc-4d83-8354-5434fc19da18,1200,Inventory,asset_ca_inv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2023-11-13 17:38:44.529581+00:00,2023-11-13 17:38:44.529591+00:00,001400050009,3,0,57cb12bd-45e7-470b-bf80-140445ad3764,6294,Garbage,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
84,2023-11-13 17:38:44.543776+00:00,2023-11-13 17:38:44.543788+00:00,001400050001,3,0,01218b4a-1cc6-4e81-8153-a34da5ff7f2e,6295,Electricity,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
85,2023-11-13 17:38:44.552585+00:00,2023-11-13 17:38:44.552599+00:00,00140005000K,3,0,8ab7033f-db7b-460f-a5fe-6fe5d6c22e4f,6300,Property Management,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
86,2023-11-13 17:38:44.556906+00:00,2023-11-13 17:38:44.556916+00:00,00140005000Y,3,0,e3c44191-37fd-4108-a16d-67ddc6814229,6400,Vacancy,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae


## Get CoA Accounts by CoA Model UUID

In [18]:
coa_accounts_by_coa_uuid_qs = entity_model.get_coa_accounts(coa_model=default_coa_model.uuid)
pd.DataFrame(coa_accounts_by_coa_uuid_qs.values())

,created,updated,path,depth,numchild,uuid,code,name,role,role_default,balance_type,locked,active,coa_model_id
0,2023-11-13 17:38:44.234131+00:00,2023-11-13 17:38:44.234138+00:00,001400010009,3,0,690baf2c-e77f-4f0e-b4fb-46eb02696410,1010,Cash,asset_ca_cash,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
1,2023-11-13 17:38:44.240975+00:00,2023-11-13 17:38:44.240981+00:00,001400010005,3,0,2f4fd01d-8194-4c78-b7ae-7dac3627f438,1050,Short Term Investments,asset_ca_mkt_sec,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
2,2023-11-13 17:38:44.248853+00:00,2023-11-13 17:38:44.248859+00:00,00140001000O,3,0,f6de4cef-abfa-416d-b115-42eab4290a4b,1100,Accounts Receivable,asset_ca_recv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
3,2023-11-13 17:38:44.251788+00:00,2023-11-13 17:38:44.251794+00:00,00140001000B,3,0,6d556011-8e89-4b27-9eea-5f7f33478238,1110,Uncollectibles,asset_ca_uncoll,True,credit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
4,2023-11-13 17:38:44.237139+00:00,2023-11-13 17:38:44.237146+00:00,001400010002,3,0,12ed5dee-51dc-4d83-8354-5434fc19da18,1200,Inventory,asset_ca_inv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2023-11-13 17:38:44.529581+00:00,2023-11-13 17:38:44.529591+00:00,001400050009,3,0,57cb12bd-45e7-470b-bf80-140445ad3764,6294,Garbage,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
84,2023-11-13 17:38:44.543776+00:00,2023-11-13 17:38:44.543788+00:00,001400050001,3,0,01218b4a-1cc6-4e81-8153-a34da5ff7f2e,6295,Electricity,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
85,2023-11-13 17:38:44.552585+00:00,2023-11-13 17:38:44.552599+00:00,00140005000K,3,0,8ab7033f-db7b-460f-a5fe-6fe5d6c22e4f,6300,Property Management,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
86,2023-11-13 17:38:44.556906+00:00,2023-11-13 17:38:44.556916+00:00,00140005000Y,3,0,e3c44191-37fd-4108-a16d-67ddc6814229,6400,Vacancy,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae


## Get CoA Accounts by CoA Model Slug

In [19]:
coa_accounts_by_coa_slug_qs = entity_model.get_coa_accounts(coa_model=default_coa_model.slug)
pd.DataFrame(coa_accounts_by_coa_slug_qs.values())

,created,updated,path,depth,numchild,uuid,code,name,role,role_default,balance_type,locked,active,coa_model_id
0,2023-11-13 17:38:44.234131+00:00,2023-11-13 17:38:44.234138+00:00,001400010009,3,0,690baf2c-e77f-4f0e-b4fb-46eb02696410,1010,Cash,asset_ca_cash,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
1,2023-11-13 17:38:44.240975+00:00,2023-11-13 17:38:44.240981+00:00,001400010005,3,0,2f4fd01d-8194-4c78-b7ae-7dac3627f438,1050,Short Term Investments,asset_ca_mkt_sec,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
2,2023-11-13 17:38:44.248853+00:00,2023-11-13 17:38:44.248859+00:00,00140001000O,3,0,f6de4cef-abfa-416d-b115-42eab4290a4b,1100,Accounts Receivable,asset_ca_recv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
3,2023-11-13 17:38:44.251788+00:00,2023-11-13 17:38:44.251794+00:00,00140001000B,3,0,6d556011-8e89-4b27-9eea-5f7f33478238,1110,Uncollectibles,asset_ca_uncoll,True,credit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
4,2023-11-13 17:38:44.237139+00:00,2023-11-13 17:38:44.237146+00:00,001400010002,3,0,12ed5dee-51dc-4d83-8354-5434fc19da18,1200,Inventory,asset_ca_inv,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2023-11-13 17:38:44.529581+00:00,2023-11-13 17:38:44.529591+00:00,001400050009,3,0,57cb12bd-45e7-470b-bf80-140445ad3764,6294,Garbage,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
84,2023-11-13 17:38:44.543776+00:00,2023-11-13 17:38:44.543788+00:00,001400050001,3,0,01218b4a-1cc6-4e81-8153-a34da5ff7f2e,6295,Electricity,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
85,2023-11-13 17:38:44.552585+00:00,2023-11-13 17:38:44.552599+00:00,00140005000K,3,0,8ab7033f-db7b-460f-a5fe-6fe5d6c22e4f,6300,Property Management,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
86,2023-11-13 17:38:44.556906+00:00,2023-11-13 17:38:44.556916+00:00,00140005000Y,3,0,e3c44191-37fd-4108-a16d-67ddc6814229,6400,Vacancy,ex_regular,None,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae


## Get Accounts With Codes and CoA Model

In [20]:
coa_accounts_by_codes_qs = entity_model.get_accounts_with_codes(code_list=['1010', '1050'])
pd.DataFrame(coa_accounts_by_codes_qs.values())

,created,updated,path,depth,numchild,uuid,code,name,role,role_default,balance_type,locked,active,coa_model_id
0,2023-11-13 17:38:44.234131+00:00,2023-11-13 17:38:44.234138+00:00,001400010009,3,0,690baf2c-e77f-4f0e-b4fb-46eb02696410,1010,Cash,asset_ca_cash,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae
1,2023-11-13 17:38:44.240975+00:00,2023-11-13 17:38:44.240981+00:00,001400010005,3,0,2f4fd01d-8194-4c78-b7ae-7dac3627f438,1050,Short Term Investments,asset_ca_mkt_sec,True,debit,False,True,f1262cc2-82d1-4ed1-b19b-6e0f241ef1ae


## Create Account Model

In [21]:
coa_model, account_model = entity_model.create_account(
    coa_model=another_coa_model,
    account_model_kwargs={
        'code': f'1{str(randint(10000, 99999))}ABC',
        'role': roles.ASSET_CA_INVENTORY,
        'name': 'A cool account created from the EntityModel API!',
        'balance_type': roles.DEBIT,
        'active': True
    })

In [22]:
account_model

<AccountModel: ASSETS - 135062ABC: A cool account created from the EntityModel API! (ASSET_CA_INV/debit)>

In [23]:
another_coa_accounts_qs = entity_model.get_coa_accounts(coa_model=another_coa_model)
pd.DataFrame(another_coa_accounts_qs.values())

,created,updated,path,depth,numchild,uuid,code,name,role,role_default,balance_type,locked,active,coa_model_id
0,2023-11-13 17:38:48.539100+00:00,2023-11-13 17:38:48.539113+00:00,001100060001,3,0,22a76645-060b-4d73-a29f-e2073afc7691,135062ABC,A cool account created from the EntityModel API!,asset_ca_inv,None,debit,False,True,69446d15-9053-4406-9345-0b75d919cbe2


# Customers

## Get Customers

In [24]:
customer_qs = entity_model.get_customers()
pd.DataFrame(customer_qs.values())

,created,updated,address_1,address_2,city,state,zip_code,country,email,website,phone,sales_tax_rate,uuid,customer_name,customer_number,entity_model_id,description,active,hidden,additional_info
0,2023-11-13 17:38:44.676474+00:00,2023-11-13 17:38:44.676484+00:00,7488 Smith Gateway Suite 122Circle,21895,West Michaelfort,WI,43322,US,vmatthews@example.org,http://ross.com/,290-935-1566,0.0,46d934b0-093e-4639-968d-7f1574e5f784,James Luna,C-0000000001,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
1,2023-11-13 17:38:44.678245+00:00,2023-11-13 17:38:44.678252+00:00,81190 Baxter ParkDrives,None,Barkertown,AZ,20070,US,aaron95@example.net,https://brown.com/,001-783-913-7585,0.0,e81399e3-35c7-4a3f-a214-33dade639b70,Ronnie Holt,C-0000000002,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
2,2023-11-13 17:38:44.680203+00:00,2023-11-13 17:38:44.680213+00:00,4109 Gabrielle Mission Suite 772Grove,18361,East Tina,MH,58893,US,annette39@example.org,http://www.reed.com/,832.453.8416x042,0.0,8366a9b4-0192-420d-9e6b-2df1912cd054,Christine Flores,C-0000000003,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
3,2023-11-13 17:38:44.681891+00:00,2023-11-13 17:38:44.681899+00:00,88720 John PlainsNeck,402,South Samantha,NE,57976,US,alexaespinoza@example.com,http://www.thomas.com/,287-577-2407x9722,0.0,babcb385-7b6e-4dbb-ae3b-6710aa29f2cb,Mrs. Lisa Moore,C-0000000004,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
4,2023-11-13 17:38:44.683906+00:00,2023-11-13 17:38:44.683912+00:00,9100 Wright Ranch Suite 384Pike,60754,Gardnerburgh,NJ,48578,US,akramer@example.com,https://www.hale.com/,001-847-371-6965x790,0.0,d74e5118-8608-4b5f-a97b-5cee198b077b,Michelle Watkins,C-0000000005,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
5,2023-11-13 17:38:44.685442+00:00,2023-11-13 17:38:44.685449+00:00,75184 Hebert BrooksMotorway,None,Victoriachester,WA,94396,US,johnwilson@example.com,https://farmer.com/,001-870-501-2293,0.0,58965fd4-107f-4c70-a985-990fd21a1b74,Kristy Williams,C-0000000006,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
6,2023-11-13 17:38:44.687528+00:00,2023-11-13 17:38:44.687534+00:00,1668 Charles Spring Apt. 303Field,938,Lake Blakefort,AK,03508,US,henrylee@example.com,http://www.adkins.com/,6833342458,0.0,ac6214dd-7333-445a-bef1-56f4f180fb99,Joshua Patel,C-0000000007,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
7,2023-11-13 17:38:44.689018+00:00,2023-11-13 17:38:44.689025+00:00,701 Joyce StreamRadial,7235,East Stevenstad,AZ,20116,US,ubrown@example.com,http://www.kennedy-parker.net/,940-372-6345,0.0,91594ec2-1654-490e-ba29-4305ff99e32c,Karen Branch,C-0000000008,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
8,2023-11-13 17:38:44.690531+00:00,2023-11-13 17:38:44.690537+00:00,5039 Levi ShoalGrove,None,Butlerborough,OH,20885,US,michael91@example.com,https://jones.com/,4106464993,0.0,cfbc50a3-1b0f-4c9c-8e45-2ed23ac2e28f,Patrick Frye,C-0000000009,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}
9,2023-11-13 17:38:44.691993+00:00,2023-11-13 17:38:44.691999+00:00,4510 Moreno Passage Apt. 781Fall,0187,Hendersonland,NE,99911,US,rgraham@example.com,http://gross.com/,297-845-6754,0.0,1e262888-3241-48da-8dcd-9df51028451e,Stephanie Hebert,C-0000000010,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool customer description. We love customers!,True,False,{}


## Create Customers

In [25]:
customer_model = entity_model.create_customer(customer_model_kwargs={
    'customer_name': 'Mr. Big',
    'description': 'A great paying customer!',
})

# Vendors

## Get Vendors

In [26]:
vendor_qs = entity_model.get_vendors()
pd.DataFrame(vendor_qs.values())

,created,updated,address_1,address_2,city,state,zip_code,country,email,website,...,account_type,tax_id_number,uuid,vendor_number,vendor_name,entity_model_id,description,active,hidden,additional_info
0,2023-11-13 17:38:44.673404+00:00,2023-11-13 17:38:44.673420+00:00,3714 Brown Isle Suite 265,None,Lake Nicolemouth,VI,25263,US,thomasfrost@example.net,http://duke.com/,...,checking,None,2f1b986f-9457-4e2f-9f15-660506f881bd,V-0000000001,Garcia-Holt,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
1,2023-11-13 17:38:44.673453+00:00,2023-11-13 17:38:44.673457+00:00,78760 Daniel Club Apt. 355,None,Lake Oliviaview,TN,21147,US,thompsonjennifer@example.com,http://www.hopkins.biz/,...,checking,None,29f2ddb4-3b48-44cd-b5a7-394a715a0f7d,V-0000000002,Wilkinson-Briggs,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
2,2023-11-13 17:38:44.673476+00:00,2023-11-13 17:38:44.673480+00:00,16070 Cooley Springs,None,Georgeside,LA,91499,US,robertchang@example.net,http://www.miller-jackson.com/,...,checking,None,eefe8979-eb21-4cd3-bfdb-c960727cd190,V-0000000003,"Johnson, Bailey and Terry",44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
3,2023-11-13 17:38:44.673496+00:00,2023-11-13 17:38:44.673500+00:00,98138 Booth Ports,3260,Michelleton,VT,92327,US,carlwilson@example.net,https://rice-smith.com/,...,checking,None,61fd995c-26f6-4826-8c27-30fb2436bb45,V-0000000004,"Simpson, Yoder and Price",44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
4,2023-11-13 17:38:44.673516+00:00,2023-11-13 17:38:44.673520+00:00,2708 Chen Rest,8860,New Virginiaborough,CT,23115,US,wilsonjudith@example.com,https://trujillo.com/,...,checking,None,29d24570-8299-404b-809c-bc6f86f1a080,V-0000000005,Griffin-Rose,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
5,2023-11-13 17:38:44.673536+00:00,2023-11-13 17:38:44.673540+00:00,605 Watson Parkways Suite 053,None,New Shawnshire,AK,50146,US,tina15@example.com,https://mcclure.com/,...,checking,None,74182bf9-39b9-41dc-88dd-f2a050d75a87,V-0000000006,Hernandez-Roth,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
6,2023-11-13 17:38:44.673572+00:00,2023-11-13 17:38:44.673576+00:00,17420 Tracy Turnpike,None,Jessicatown,VA,33488,US,david95@example.org,http://www.johnson.com/,...,checking,None,8f49980e-f408-4d0d-87ef-105be0aa9719,V-0000000007,Adam Boyd,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
7,2023-11-13 17:38:44.673593+00:00,2023-11-13 17:38:44.673597+00:00,89500 Hill Branch,None,West Dylanland,MT,11787,US,davissamantha@example.com,https://www.peterson.com/,...,checking,None,1f9894ea-c7a3-4110-8bab-d043509bc7dc,V-0000000008,Tanner Miller,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
8,2023-11-13 17:38:44.673613+00:00,2023-11-13 17:38:44.673616+00:00,62484 Danielle Canyon Apt. 283,None,Taylorside,WA,66535,US,garrettmclean@example.org,https://www.thompson.com/,...,checking,None,0a52ef1f-ba8c-459f-8254-8113a87730b5,V-0000000009,Kara Schroeder,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}
9,2023-11-13 17:38:44.673631+00:00,2023-11-13 17:38:44.673634+00:00,511 Clark Unions,None,Lake Jamesfort,MA,05065,US,pereznathan@example.org,http://boyd.com/,...,checking,None,13e8805b-6b12-4a89-966e-b44fd5deb769,V-0000000010,Munoz-Thomas,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,A cool vendor description.,True,False,{}


## Create Vendor

In [27]:
vendor_model = entity_model.create_vendor(vendor_model_kwargs={
    'vendor_name': 'ACME LLC',
    'description': 'A Reliable Vendor!'
})

# Invoices

## Get Invoices

In [28]:
invoices_qs = entity_model.get_invoices()
pd.DataFrame(invoices_qs.values())

,created,updated,amount_due,amount_paid,amount_receivable,amount_unearned,amount_earned,accrue,progress,ledger_id,...,prepaid_account_id,unearned_account_id,additional_info,ce_model_id,date_draft,date_in_review,date_approved,date_paid,date_void,date_canceled
0,2023-11-13 17:38:47.777142+00:00,2023-11-13 17:38:47.783719+00:00,627.83,0.00,0.00,0.00,0.00,True,0.00,0b60c048-9bd1-4c4f-a42a-b6c0e546ea86,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2023-03-07,2023-03-11,None,None,None,None
1,2023-11-13 17:38:47.766057+00:00,2023-11-13 17:38:47.773837+00:00,45.90,0.00,0.00,0.00,0.00,True,0.00,1a245dfe-6210-4368-af0b-3fcfe2ccba0b,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2022-10-11,2022-10-19,None,None,None,None
2,2023-11-13 17:38:47.712394+00:00,2023-11-13 17:38:47.726722+00:00,543.80,543.80,0.00,0.00,543.80,True,1.00,ca841cd3-7e03-4fc3-a832-c8a02bf8ef46,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2023-03-04,2023-03-05,2023-03-07,2023-03-15,None,None
3,2023-11-13 17:38:47.700360+00:00,2023-11-13 17:38:47.709055+00:00,2466.11,0.00,0.00,0.00,0.00,True,0.00,489bcbf1-aac0-4d6b-b348-5a9bc58fd067,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2023-02-18,2023-02-22,None,None,None,2023-02-26
4,2023-11-13 17:38:47.671054+00:00,2023-11-13 17:38:47.678512+00:00,1387.02,0.00,1387.02,0.00,1387.02,True,1.00,2c9639a9-5267-425b-908e-ab8e66ae9b36,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2023-03-21,2023-03-28,2023-03-30,None,None,None
5,2023-11-13 17:38:47.658698+00:00,2023-11-13 17:38:47.667152+00:00,6.74,0.00,0.00,0.00,0.00,True,0.00,0b05e3b5-339a-4d66-ace2-8fe94583aa1e,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2023-02-03,None,None,None,None,None
6,2023-11-13 17:38:47.593969+00:00,2023-11-13 17:38:47.611982+00:00,1574.04,1574.04,0.00,0.00,1574.04,True,1.00,6fee0ef6-f10c-4567-8fd0-577c13fe4c2e,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2023-01-26,2023-01-30,2023-02-05,2023-02-10,None,None
7,2023-11-13 17:38:47.580196+00:00,2023-11-13 17:38:47.590320+00:00,7512.08,0.00,0.00,0.00,0.00,True,0.00,f27bb075-f789-4a2d-adb6-21d3ec78ffe8,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2022-11-18,None,None,None,None,None
8,2023-11-13 17:38:47.566471+00:00,2023-11-13 17:38:47.576600+00:00,2227.07,0.00,0.00,0.00,0.00,True,0.00,cef4bd3f-4d4e-4ee3-a542-2e522e8a699b,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2023-01-26,2023-02-01,None,None,None,2023-02-07
9,2023-11-13 17:38:47.499200+00:00,2023-11-13 17:38:47.522744+00:00,3899.67,3899.67,0.00,0.00,3899.67,True,1.00,392cfd4b-33be-4d8d-adfb-0ca47a2d6d01,...,f6de4cef-abfa-416d-b115-42eab4290a4b,3a4b3db9-eede-47da-8107-f7efd1dd27af,{},None,2022-11-27,2022-11-28,2022-12-01,2022-12-04,None,None


## Create Invoice

In [29]:
invoice_model = entity_model.create_invoice(
    customer_model='C-0000000006',
    terms=InvoiceModel.TERMS_NET_30
)

In [30]:
invoice_model

<InvoiceModel: Invoice: I-2023-0000000015 | Draft>

## Add Items to Invoices

In [31]:
invoices_item_models = invoice_model.get_item_model_qs()

# K= number of items...
K = 6

invoice_itemtxs = {
    im.item_number: {
        'unit_cost': round(random() * 10, 2),
        'quantity': round(random() * 100, 2),
        'total_amount': None
    } for im in choices(invoices_item_models, k=K)
}

# Choose operation ITEMIZE_APPEND to append itemtxs...
invoice_itemtxs = invoice_model.migrate_itemtxs(itemtxs=invoice_itemtxs,
                                                commit=True,
                                                operation=InvoiceModel.ITEMIZE_REPLACE)
invoice_itemtxs

[<ItemTransactionModel: Invoice Model: a0d06c0e-c7df-4e21-a5a5-c7909c043ed6 | 342.61>,
 <ItemTransactionModel: Invoice Model: a0d06c0e-c7df-4e21-a5a5-c7909c043ed6 | 235.50>,
 <ItemTransactionModel: Invoice Model: a0d06c0e-c7df-4e21-a5a5-c7909c043ed6 | 404.04>,
 <ItemTransactionModel: Invoice Model: a0d06c0e-c7df-4e21-a5a5-c7909c043ed6 | 905.21>,
 <ItemTransactionModel: Invoice Model: a0d06c0e-c7df-4e21-a5a5-c7909c043ed6 | 326.02>,
 <ItemTransactionModel: Invoice Model: a0d06c0e-c7df-4e21-a5a5-c7909c043ed6 | 537.68>]

In [32]:
invoice_model.amount_due

Decimal('2751.06')

# Bills

## Get Bills

In [33]:
bills_qs = entity_model.get_bills()
pd.DataFrame(bills_qs.values())

,created,updated,amount_due,amount_paid,amount_receivable,amount_unearned,amount_earned,accrue,progress,ledger_id,...,xref,vendor_id,additional_info,ce_model_id,date_draft,date_in_review,date_approved,date_paid,date_void,date_canceled
0,2023-11-13 17:38:46.944377+00:00,2023-11-13 17:38:46.953462+00:00,2072.97,0.00,0.00,0.00,0.00,True,0.00,f1d28045-4085-4549-8bc8-00709f326f88,...,None,29d24570-8299-404b-809c-bc6f86f1a080,{},None,2023-01-01,None,None,None,None,None
1,2023-11-13 17:38:46.875707+00:00,2023-11-13 17:38:46.884190+00:00,1896.10,0.00,0.00,0.00,0.00,True,0.00,f6df493d-fe5e-4fa9-b448-bbeb1e556e5b,...,None,74182bf9-39b9-41dc-88dd-f2a050d75a87,{},None,2023-01-26,None,None,None,None,None
2,2023-11-13 17:38:46.786916+00:00,2023-11-13 17:38:46.799465+00:00,19406.11,0.00,0.00,0.00,0.00,True,0.00,d12a1f69-a8ba-42f5-ae08-8a2057445b03,...,None,0a52ef1f-ba8c-459f-8254-8113a87730b5,None,None,2023-02-03,2023-02-08,None,None,None,None
3,2023-11-13 17:38:46.724595+00:00,2023-11-13 17:38:46.749358+00:00,935.23,935.23,0.00,0.00,935.23,True,1.00,35ef9df5-87a8-479f-97d6-c18f9d28ee61,...,None,eefe8979-eb21-4cd3-bfdb-c960727cd190,{},None,2022-12-29,2023-01-03,2023-01-05,2023-01-12,None,None
4,2023-11-13 17:38:46.666424+00:00,2023-11-13 17:38:46.675674+00:00,1136.04,0.00,0.00,0.00,0.00,True,0.00,c0a217a6-dcd7-4b1f-9c8d-8cfffb09bba6,...,None,eefe8979-eb21-4cd3-bfdb-c960727cd190,{},None,2023-02-22,2023-02-27,None,None,None,None
5,2023-11-13 17:38:46.605570+00:00,2023-11-13 17:38:46.613135+00:00,710.96,0.00,0.00,0.00,0.00,True,0.00,2e474fd5-f6c0-4512-9d8a-7a09cf91f34a,...,None,74182bf9-39b9-41dc-88dd-f2a050d75a87,{},None,2023-02-12,None,None,None,None,None
6,2023-11-13 17:38:46.562688+00:00,2023-11-13 17:38:46.569390+00:00,17133.22,0.00,0.00,0.00,0.00,True,0.00,a51d3066-ff14-4015-a53e-3538c11590b1,...,None,61fd995c-26f6-4826-8c27-30fb2436bb45,None,None,2023-02-22,None,None,None,None,None
7,2023-11-13 17:38:46.526417+00:00,2023-11-13 17:38:46.534589+00:00,581.01,0.00,0.00,0.00,0.00,True,0.00,ef11db35-53c4-43b6-ac54-b75b38595bc9,...,None,eefe8979-eb21-4cd3-bfdb-c960727cd190,{},None,2023-02-05,2023-02-10,None,None,None,None
8,2023-11-13 17:38:46.459644+00:00,2023-11-13 17:38:46.465855+00:00,10.62,0.00,0.00,0.00,0.00,True,0.00,9126e569-35d7-48e6-a108-cb5b41118551,...,None,74182bf9-39b9-41dc-88dd-f2a050d75a87,{},None,2023-02-15,2023-02-18,None,None,None,None
9,2023-11-13 17:38:46.396997+00:00,2023-11-13 17:38:46.403191+00:00,503.69,0.00,0.00,0.00,0.00,True,0.00,f44cf4aa-459a-42c1-87bf-3d380c78b925,...,None,0a52ef1f-ba8c-459f-8254-8113a87730b5,{},None,2023-02-10,2023-02-13,None,None,None,None


## Create Bill

In [34]:
bill_model = entity_model.create_bill(
    vendor_model='V-0000000002',
    terms=BillModel.TERMS_NET_60
)

In [35]:
bill_model

<BillModel: Bill: B-2023-0000000023 | Draft>

## Add Items to Bills

In [36]:
bill_item_models = bill_model.get_item_model_qs()

K = 6

bill_itemtxs = {
    im.item_number: {
        'unit_cost': round(random() * 10, 2),
        'quantity': round(random() * 100, 2),
        'total_amount': None
    } for im in choices(bill_item_models, k=K)
}

# Choose operation ITEMIZE_APPEND to append itemtxs...
bill_itemtxs = bill_model.migrate_itemtxs(itemtxs=bill_itemtxs,
                                          commit=True,
                                          operation=BillModel.ITEMIZE_REPLACE)

bill_itemtxs

[<ItemTransactionModel: Bill Model: 3aee4a4e-d600-4d0e-9a8d-0e1a17d409c8 | 115.64>,
 <ItemTransactionModel: Bill Model: 3aee4a4e-d600-4d0e-9a8d-0e1a17d409c8 | 479.17>,
 <ItemTransactionModel: Bill Model: 3aee4a4e-d600-4d0e-9a8d-0e1a17d409c8 | 1.83>,
 <ItemTransactionModel: Bill Model: 3aee4a4e-d600-4d0e-9a8d-0e1a17d409c8 | 573.97>,
 <ItemTransactionModel: Bill Model: 3aee4a4e-d600-4d0e-9a8d-0e1a17d409c8 | 41.83>,
 <ItemTransactionModel: Bill Model: 3aee4a4e-d600-4d0e-9a8d-0e1a17d409c8 | 144.99>]

In [37]:
bill_model.amount_due

Decimal('1357.43')

# Purchase Orders

## Get Purchase Orders

In [38]:
purchase_orders_qs = entity_model.get_purchase_orders()
pd.DataFrame(purchase_orders_qs.values())

,created,updated,markdown_notes,uuid,po_number,po_title,po_status,po_amount,po_amount_received,entity_id,date_draft,date_in_review,date_approved,date_void,date_fulfilled,date_canceled,ce_model_id
0,2023-11-13 17:38:44.971392+00:00,2023-11-13 17:38:44.979742+00:00,None,eaeeeed7-45e0-48f6-8f9f-7bf791d3dd04,PO-2023-0000000001,PO Number PO-2023-0000000001,in_review,1521.80,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2023-03-05,2023-03-11,None,None,None,None,None
1,2023-11-13 17:38:45.067737+00:00,2023-11-13 17:38:45.077063+00:00,None,d0dab8bc-17fe-4dc4-af15-078b6803e5a7,PO-2023-0000000002,PO Number PO-2023-0000000002,in_review,5164.17,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2023-01-30,2023-02-04,None,None,None,None,None
2,2023-11-13 17:38:45.131587+00:00,2023-11-13 17:38:45.145801+00:00,None,c42f6564-4506-4ea2-a2c4-7fd18fd1c945,PO-2023-0000000003,PO Number PO-2023-0000000003,in_review,8682.90,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2023-03-19,2023-03-22,None,None,None,None,None
3,2023-11-13 17:38:45.237670+00:00,2023-11-13 17:38:45.247584+00:00,None,4e9db824-af40-4d93-9b25-03b4da6551de,PO-2023-0000000004,PO Number PO-2023-0000000004,approved,19201.04,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2023-03-19,2023-03-24,2023-04-01,None,None,None,None
4,2023-11-13 17:38:45.305146+00:00,2023-11-13 17:38:45.314249+00:00,None,28d2ca06-bdbf-4634-bfa7-4d26e716f63b,PO-2023-0000000005,PO Number PO-2023-0000000005,draft,4894.57,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2023-01-07,None,None,None,None,None,None
5,2023-11-13 17:38:45.365373+00:00,2023-11-13 17:38:45.404823+00:00,None,39ab1a83-4cd3-457f-9385-afe19e1fc781,PO-2022-0000000001,PO Number PO-2022-0000000001,fulfilled,3364.02,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2022-11-26,2022-12-03,2022-12-06,None,2022-12-11,None,None
6,2023-11-13 17:38:45.484381+00:00,2023-11-13 17:38:45.492915+00:00,None,98c5e030-c83b-4b75-9202-0b84248d6dde,PO-2023-0000000006,PO Number PO-2023-0000000006,draft,16338.32,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2023-03-16,None,None,None,None,None,None
7,2023-11-13 17:38:45.551000+00:00,2023-11-13 17:38:45.558003+00:00,None,a8c529ca-f11c-44f3-8f43-df314c168474,PO-2023-0000000007,PO Number PO-2023-0000000007,approved,4412.35,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2023-03-02,2023-03-06,2023-03-08,None,None,None,None
8,2023-11-13 17:38:45.646679+00:00,2023-11-13 17:38:45.649995+00:00,None,46403eef-bafd-477d-a659-b15d1bb3e32d,PO-2022-0000000002,PO Number PO-2022-0000000002,draft,1374.67,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2022-11-06,None,None,None,None,None,None
9,2023-11-13 17:38:45.702992+00:00,2023-11-13 17:38:45.712674+00:00,None,2fb8d512-96c9-460a-b277-ebe20c835873,PO-2023-0000000008,PO Number PO-2023-0000000008,in_review,14391.63,0.00,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,2023-03-10,2023-03-16,None,None,None,None,None


## Create Purchase Order

In [39]:
po_model = entity_model.create_purchase_order()

## Add Items to Purchase Orders

In [40]:
po_item_models = po_model.get_item_model_qs()

K = 6

po_itemtxs = {
    im.item_number: {
        'unit_cost': round(random() * 10, 2),
        'quantity': round(random() * 100, 2),
        'total_amount': None
    } for im in choices(po_item_models, k=K)
}

# Choose operation ITEMIZE_APPEND to append itemtxs...
po_itemtxs = po_model.migrate_itemtxs(itemtxs=po_itemtxs,
                                      commit=True,
                                      operation=EstimateModel.ITEMIZE_REPLACE)

po_itemtxs

[<ItemTransactionModel: PO Model: e6102e35-491a-4fd8-8892-5eb12360621e | Not Ordered | 40.36>,
 <ItemTransactionModel: PO Model: e6102e35-491a-4fd8-8892-5eb12360621e | Not Ordered | 319.77>,
 <ItemTransactionModel: PO Model: e6102e35-491a-4fd8-8892-5eb12360621e | Not Ordered | 49.02>,
 <ItemTransactionModel: PO Model: e6102e35-491a-4fd8-8892-5eb12360621e | Not Ordered | 21.62>,
 <ItemTransactionModel: PO Model: e6102e35-491a-4fd8-8892-5eb12360621e | Not Ordered | 217.84>,
 <ItemTransactionModel: PO Model: e6102e35-491a-4fd8-8892-5eb12360621e | Not Ordered | 475.93>]

In [41]:
po_model.po_amount

Decimal('1124.54')

# Estimates/Contracts

## Get Estimates/Contracts

In [42]:
estimates_qs = entity_model.get_estimates()
pd.DataFrame(estimates_qs.values())

,created,updated,markdown_notes,uuid,estimate_number,entity_id,customer_id,terms,title,status,...,date_in_review,date_approved,date_completed,date_canceled,date_void,revenue_estimate,labor_estimate,material_estimate,equipment_estimate,other_estimate
0,2023-11-13 17:38:46.886888+00:00,2023-11-13 17:38:46.897893+00:00,None,6c1593e5-9e07-40f1-807a-5bb5d3c4661f,E-2023-0000000025,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,8366a9b4-0192-420d-9e6b-2df1912cd054,target,Customer Estimate 2023-01-01 00:00:00+00:00,completed,...,2023-01-04,2023-01-06,2023-01-11,None,None,800.68,0.00,0.00,0.00,201.22
1,2023-11-13 17:38:46.763741+00:00,2023-11-13 17:38:46.770160+00:00,None,ac203317-1259-4035-8d79-f9438f2cb7fb,E-2023-0000000024,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,1e262888-3241-48da-8dcd-9df51028451e,t&m,Customer Estimate 2023-01-26 00:00:00+00:00,in_review,...,2023-01-26,None,None,None,None,1056.77,0.00,0.00,0.00,640.70
2,2023-11-13 17:38:46.677547+00:00,2023-11-13 17:38:46.684864+00:00,None,fade3523-3695-45ff-97b1-3dd1409a3b0e,E-2023-0000000023,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,b51a5ef6-1c51-4115-bb07-1e6b11262a27,fixed,Customer Estimate 2022-12-29 00:00:00+00:00,completed,...,2022-12-29,2023-01-02,2023-01-05,None,None,757.37,0.00,0.00,0.00,388.41
3,2023-11-13 17:38:46.615297+00:00,2023-11-13 17:38:46.621651+00:00,None,9d92967c-8c09-4758-82f0-02bdc5d52297,E-2023-0000000022,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,8366a9b4-0192-420d-9e6b-2df1912cd054,t&m,Customer Estimate 2023-02-22 00:00:00+00:00,canceled,...,2023-03-01,None,None,2023-03-04,None,425.90,0.00,0.00,0.00,372.38
4,2023-11-13 17:38:46.536798+00:00,2023-11-13 17:38:46.547460+00:00,None,68833f6b-f74b-4879-8ab2-c38ba151f550,E-2023-0000000021,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,8366a9b4-0192-420d-9e6b-2df1912cd054,fixed,Customer Estimate 2023-02-12 00:00:00+00:00,in_review,...,2023-02-12,None,None,None,None,2795.21,0.00,0.00,0.00,1303.50
5,2023-11-13 17:38:46.468053+00:00,2023-11-13 17:38:46.480791+00:00,None,2c2baece-b0d8-4256-a55a-2e0326969cdd,E-2023-0000000020,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,5da00d3e-c9ab-4ffe-a022-abb6f044c772,other,Customer Estimate 2023-02-05 00:00:00+00:00,completed,...,2023-02-12,2023-02-13,2023-02-20,None,None,2205.25,0.00,0.00,0.00,1413.63
6,2023-11-13 17:38:46.404943+00:00,2023-11-13 17:38:46.416423+00:00,None,7a0930dd-673e-47d9-88d8-1928eaf85207,E-2023-0000000019,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,91594ec2-1654-490e-ba29-4305ff99e32c,t&m,Customer Estimate 2023-02-15 00:00:00+00:00,completed,...,2023-02-22,2023-02-28,2023-03-08,None,None,2048.06,0.00,0.00,0.00,1848.24
7,2023-11-13 17:38:46.350434+00:00,2023-11-13 17:38:46.355283+00:00,None,19fbbed0-a18b-45a8-8c14-3c96f9435acc,E-2023-0000000018,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,ac6214dd-7333-445a-bef1-56f4f180fb99,target,Customer Estimate 2023-02-10 00:00:00+00:00,draft,...,None,None,None,None,None,1814.88,0.00,0.00,0.00,640.96
8,2023-11-13 17:38:46.252139+00:00,2023-11-13 17:38:46.260645+00:00,None,bab2707c-cdb0-4c09-a464-8500a3f3fc92,E-2023-0000000017,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,46d934b0-093e-4639-968d-7f1574e5f784,fixed,Customer Estimate 2022-10-16 00:00:00+00:00,in_review,...,2022-10-19,None,None,None,None,1791.28,0.00,0.00,0.00,998.99
9,2023-11-13 17:38:46.197362+00:00,2023-11-13 17:38:46.204714+00:00,None,59fd0523-3a5d-4f4c-ac81-f80c5c400635,E-2023-0000000016,44d57e0b-48bc-4e1b-8d9f-04b781cc607c,d74e5118-8608-4b5f-a97b-5cee198b077b,fixed,Customer Estimate 2022-10-10 00:00:00+00:00,completed,...,2022-10-13,2022-10-15,2022-10-23,None,None,1012.00,0.00,0.00,0.00,73.18


## Create Estimate

In [43]:
estimate_model = entity_model.create_estimate(
    estimate_title='A quote for new potential customer!',
    customer_model='C-0000000009',
    contract_terms=EstimateModel.CONTRACT_TERMS_FIXED
)

## Add Items to Estimates

In [44]:
estimate_item_models = estimate_model.get_item_model_qs()

K = 6

estimate_itemtxs = {
    im.item_number: {
        'unit_cost': round(random() * 10, 2),
        'unit_revenue': round(random() * 20, 2),
        'quantity': round(random() * 100, 2),
        'total_amount': None
    } for im in choices(estimate_item_models, k=K)
}

# Choose operation ITEMIZE_APPEND to append itemtxs...
estimate_itemtxs = estimate_model.migrate_itemtxs(itemtxs=estimate_itemtxs,
                                                  commit=True,
                                                  operation=EstimateModel.ITEMIZE_REPLACE)

estimate_itemtxs

[<ItemTransactionModel: Estimate/Contract Model: 10d40a18-c411-4be5-a0f9-f03fa5fca71b | 743.82>,
 <ItemTransactionModel: Estimate/Contract Model: 10d40a18-c411-4be5-a0f9-f03fa5fca71b | 62.15>,
 <ItemTransactionModel: Estimate/Contract Model: 10d40a18-c411-4be5-a0f9-f03fa5fca71b | 117.56>,
 <ItemTransactionModel: Estimate/Contract Model: 10d40a18-c411-4be5-a0f9-f03fa5fca71b | 14.38>,
 <ItemTransactionModel: Estimate/Contract Model: 10d40a18-c411-4be5-a0f9-f03fa5fca71b | 115.38>,
 <ItemTransactionModel: Estimate/Contract Model: 10d40a18-c411-4be5-a0f9-f03fa5fca71b | 56.88>]

In [45]:
estimate_model.get_cost_estimate()

Decimal('1110.17')

In [46]:
estimate_model.get_revenue_estimate()

Decimal('1064.512400000000017286083675')

In [47]:
estimate_model.get_profit_estimate()

Decimal('-45.657599999999982713916325')

In [48]:
estimate_model.get_gross_margin_estimate(as_percent=True)

-4.1126674293126335

# Bank Accounts

## Get Bank Accounts

In [49]:
bank_accounts_qs = entity_model.get_bank_accounts()
pd.DataFrame(bank_accounts_qs.values())

,created,updated,account_number,routing_number,aba_number,swift_number,account_type,uuid,name,entity_model_id,cash_account_id,active,hidden
0,2023-11-13 17:38:44.724753+00:00,2023-11-13 17:38:44.724761+00:00,008676280,002567115,CFBYGBPYC9O,None,checking,cdb95324-0836-471c-b5ab-cf364b3ae59d,"One Big Company, LLC Checking Account",44d57e0b-48bc-4e1b-8d9f-04b781cc607c,690baf2c-e77f-4f0e-b4fb-46eb02696410,True,False
1,2023-11-13 17:38:44.724776+00:00,2023-11-13 17:38:44.724780+00:00,006027853,002990223,CUVLGB3UXP9,None,savings,97efa8db-d0f3-48f3-a4d4-62e688f689ae,"One Big Company, LLC Savings Account",44d57e0b-48bc-4e1b-8d9f-04b781cc607c,690baf2c-e77f-4f0e-b4fb-46eb02696410,True,False


## Create Bank Account

In [50]:
bank_account_model = entity_model.create_bank_account(name='A big bank account!',
                                                      account_type='checking')

# Items

## Unit of Measures

### Get Unit of Measures

In [51]:
uom_qs = entity_model.get_uom_all()
pd.DataFrame(uom_qs.values())

,created,updated,uuid,name,unit_abbr,is_active,entity_id
0,2023-11-13 17:38:44.729235+00:00,2023-11-13 17:38:44.729239+00:00,0b4fd4cb-3573-481a-a746-9e0a769bfce8,Pound,lb,True,44d57e0b-48bc-4e1b-8d9f-04b781cc607c
1,2023-11-13 17:38:44.729214+00:00,2023-11-13 17:38:44.729218+00:00,924d4b04-28a5-4ae9-9eba-fae474227567,Linear Feet,ln-ft,True,44d57e0b-48bc-4e1b-8d9f-04b781cc607c
2,2023-11-13 17:38:44.729255+00:00,2023-11-13 17:38:44.729258+00:00,01851ed2-a516-43df-9f8e-02e0cd93b11d,Man Hour,man-hour,True,44d57e0b-48bc-4e1b-8d9f-04b781cc607c
3,2023-11-13 17:38:44.729245+00:00,2023-11-13 17:38:44.729249+00:00,220735b7-7c99-485e-ac8d-70144eceb6aa,Pallet,pallet,True,44d57e0b-48bc-4e1b-8d9f-04b781cc607c
4,2023-11-13 17:38:44.729225+00:00,2023-11-13 17:38:44.729229+00:00,536d0221-926b-4250-bed3-81e7bae47238,Square Fee t,sq-ft,True,44d57e0b-48bc-4e1b-8d9f-04b781cc607c
5,2023-11-13 17:38:44.729193+00:00,2023-11-13 17:38:44.729202+00:00,94cf9411-f494-4d01-8c20-2f57b5e5fd6c,Unit,unit,True,44d57e0b-48bc-4e1b-8d9f-04b781cc607c


### Create a UOM

In [52]:
uom_model_ft = entity_model.create_uom(
    name='Linear Feet',
    unit_abbr='lin-ft'
)

### Get Some UoMs

In [53]:
uom_model_unit = uom_qs.get(unit_abbr__exact='unit')
uom_model_man_hr = uom_qs.get(unit_abbr__exact='man-hour')

## Expenses

### Get Expense Items

In [54]:
expenses_qs = entity_model.get_items_expenses()
pd.DataFrame(expenses_qs.values())

,created,updated,uuid,name,item_id,item_number,item_role,item_type,uom_id,sku,...,is_product_or_service,sold_as_unit,inventory_account_id,inventory_received,inventory_received_value,cogs_account_id,earnings_account_id,expense_account_id,additional_info,entity_id
0,2023-11-13 17:38:44.938284+00:00,2023-11-13 17:38:44.938317+00:00,93227d17-61c5-48ca-a79f-2191de9ee767,Expense Item 7490,EqZXBQGQt1WYWKtXzLpf,IEX-0000000100,expense,M,924d4b04-28a5-4ae9-9eba-fae474227567,738CVCMVMF7S,...,False,False,None,None,None,None,None,ea54e0ba-9242-4e67-88c9-cd2809ef55b3,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
1,2023-11-13 17:38:44.938372+00:00,2023-11-13 17:38:44.938377+00:00,f89eb52c-7de9-459b-9bbd-ef2167280f22,Expense Item 1439,Hwo5aeE69ws5A1LpZtSQ,IEX-0000000101,expense,E,924d4b04-28a5-4ae9-9eba-fae474227567,5W1IXMC2IHR6,...,False,False,None,None,None,None,None,ac651a90-4106-47e0-843c-216d6dbc1a94,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
2,2023-11-13 17:38:44.938396+00:00,2023-11-13 17:38:44.938399+00:00,181bb12b-3c0e-4011-90fe-7c068c0d3e81,Expense Item 9391,zFqLm1BIscXSEk9xXtqW,IEX-0000000102,expense,L,220735b7-7c99-485e-ac8d-70144eceb6aa,E89IAJR0YQ46,...,False,False,None,None,None,None,None,d0e4235a-b99f-4c60-b338-56a0d244ae9f,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
3,2023-11-13 17:38:44.938417+00:00,2023-11-13 17:38:44.938421+00:00,465b827e-df04-42ad-8456-3de3cc8306e6,Expense Item 6178,TMP12KizrlcqkVdV7ArO,IEX-0000000103,expense,O,924d4b04-28a5-4ae9-9eba-fae474227567,HUOZMLYTMC1W,...,False,False,None,None,None,None,None,ed4d39a3-81c9-45ed-8cea-8be48eadfef5,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
4,2023-11-13 17:38:44.938438+00:00,2023-11-13 17:38:44.938441+00:00,21d6d955-52d9-4e13-bf1d-36ef06125070,Expense Item 5030,14yZDwD6GaytLXcg4ybG,IEX-0000000104,expense,S,536d0221-926b-4250-bed3-81e7bae47238,VZUE8SBWJW83,...,False,False,None,None,None,None,None,dcf25f98-6dfe-471f-811f-1e997bca4da4,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
5,2023-11-13 17:38:44.938458+00:00,2023-11-13 17:38:44.938461+00:00,81eaa54b-daca-4130-8834-15058adaffc8,Expense Item 1473,1sPH1A7V64XCpKBUAaCA,IEX-0000000105,expense,O,536d0221-926b-4250-bed3-81e7bae47238,8APW9AFBXU54,...,False,False,None,None,None,None,None,79179efc-836c-4f45-8975-34a736353040,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
6,2023-11-13 17:38:44.938478+00:00,2023-11-13 17:38:44.938481+00:00,8bffe9e9-2574-40aa-94df-b5c4a165bb1e,Expense Item 7721,LzuaUIIttjIIgFP0t8P7,IEX-0000000106,expense,M,924d4b04-28a5-4ae9-9eba-fae474227567,VQ43AXWT5TWH,...,False,False,None,None,None,None,None,bd5b2130-c2fa-4eb9-a0d0-052b9b4d8379,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
7,2023-11-13 17:38:44.938498+00:00,2023-11-13 17:38:44.938501+00:00,e218f380-fca0-46e8-991d-e7bb09d84997,Expense Item 6807,cP8ZDKkZmGfElvmfIJvD,IEX-0000000107,expense,L,01851ed2-a516-43df-9f8e-02e0cd93b11d,DTPVBX45ZO9Y,...,False,False,None,None,None,None,None,615a3f01-f751-43fb-bda8-0ff473c71b3e,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
8,2023-11-13 17:38:44.938517+00:00,2023-11-13 17:38:44.938521+00:00,7563a28b-3d9a-416e-a1fd-438394f420f1,Expense Item 4520,rb3wJ7TMyQX7ZMOhx2rq,IEX-0000000108,expense,E,536d0221-926b-4250-bed3-81e7bae47238,QMPZMKHD51BM,...,False,False,None,None,None,None,None,8ab7033f-db7b-460f-a5fe-6fe5d6c22e4f,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
9,2023-11-13 17:38:44.938537+00:00,2023-11-13 17:38:44.938541+00:00,81ec1475-c533-40a1-a8d4-42c0955e4b08,Expense Item 7198,HagRAJm2zssExFNPnl4D,IEX-0000000109,expense,L,0b4fd4cb-3573-481a-a746-9e0a769bfce8,17TW985AWTBQ,...,False,False,None,None,None,None,None,eacea730-3bdc-4d2a-bc94-7f35bc0a44ee,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c


### Create Expense Item

In [55]:
expense_item_model = entity_model.create_item_expense(
    name='Premium Pencils',
    uom_model=uom_model_unit,
    expense_type=ItemModel.ITEM_TYPE_MATERIAL
)

In [56]:
expense_item_model.is_expense()

True

## Services

### Get Service Items

In [57]:
services_qs = entity_model.get_items_services()
pd.DataFrame(services_qs.values())

,created,updated,uuid,name,item_id,item_number,item_role,item_type,uom_id,sku,...,is_product_or_service,sold_as_unit,inventory_account_id,inventory_received,inventory_received_value,cogs_account_id,earnings_account_id,expense_account_id,additional_info,entity_id
0,2023-11-13 17:38:44.836006+00:00,2023-11-13 17:38:44.836020+00:00,ccc94bf5-cef1-4fff-98b3-e072084adbd9,Service #3202,WDTaZ8CYfQ8jnNhOOlXH,IPR-0000000023,service,L,94cf9411-f494-4d01-8c20-2f57b5e5fd6c,N7SM2RZXZUDO,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
1,2023-11-13 17:38:44.836056+00:00,2023-11-13 17:38:44.836060+00:00,eaafc301-6c52-4bae-a807-33371128a313,Service #2361,1WzwMkYHYe8seIwvmRq9,IPR-0000000024,service,L,924d4b04-28a5-4ae9-9eba-fae474227567,QCUDIPWWP5SU,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
2,2023-11-13 17:38:44.836079+00:00,2023-11-13 17:38:44.836082+00:00,8944b270-b7ee-496d-90f1-ae20be472bc8,Service #4498,eeTg98JNMyw6mPaKarhJ,IPR-0000000025,service,L,220735b7-7c99-485e-ac8d-70144eceb6aa,V4AQHTTPL4T0,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
3,2023-11-13 17:38:44.836102+00:00,2023-11-13 17:38:44.836107+00:00,0a3df58f-f160-462c-9f15-034f3d222239,Service #4899,I2cvlF80kRaWI9CFEYSL,IPR-0000000026,service,L,924d4b04-28a5-4ae9-9eba-fae474227567,PY3VH7VMHA7U,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
4,2023-11-13 17:38:44.836129+00:00,2023-11-13 17:38:44.836133+00:00,52ba1ce4-e1c4-430a-bf09-b874102543dc,Service #4744,ohcPf56ETpTifE0bzvi5,IPR-0000000027,service,L,01851ed2-a516-43df-9f8e-02e0cd93b11d,6CJRIVQKO4GI,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
5,2023-11-13 17:38:44.836151+00:00,2023-11-13 17:38:44.836154+00:00,941e6090-2e70-4643-96e7-b604eae311fb,Service #3505,bXjdR9xEtfRHekKtt2gd,IPR-0000000028,service,L,94cf9411-f494-4d01-8c20-2f57b5e5fd6c,26B58GYTMVO8,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
6,2023-11-13 17:38:44.836171+00:00,2023-11-13 17:38:44.836174+00:00,7da36295-d896-4f75-85af-85e3946d1778,Service #9652,vzT4DGzauwChFPrONiFe,IPR-0000000029,service,L,94cf9411-f494-4d01-8c20-2f57b5e5fd6c,0WE8L2AKJHSK,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
7,2023-11-13 17:38:44.836191+00:00,2023-11-13 17:38:44.836194+00:00,176eb1a1-ad3f-4211-8a73-f4d958cebfef,Service #4110,zTmufDWCWHTyMGcG9xFp,IPR-0000000030,service,L,536d0221-926b-4250-bed3-81e7bae47238,FMMYT80ZSQFG,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
8,2023-11-13 17:38:44.836211+00:00,2023-11-13 17:38:44.836215+00:00,7029bb4d-1676-4fd9-84b5-3ee68722869a,Service #1936,IGiinlMOrl1zdfzx4nS2,IPR-0000000031,service,L,536d0221-926b-4250-bed3-81e7bae47238,RVV9KYYGC6Y0,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
9,2023-11-13 17:38:44.836232+00:00,2023-11-13 17:38:44.836235+00:00,61c54f40-1147-4159-9cde-8ecb615b1a97,Service #5387,eOWGNwo1I1gC9cXX9iqP,IPR-0000000032,service,L,0b4fd4cb-3573-481a-a746-9e0a769bfce8,JCWP5HHBA5PR,...,True,False,None,None,None,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c


### Create Service Item

In [58]:
service_model = entity_model.create_item_service(
    name='Yoga Class',
    uom_model=uom_model_man_hr
)

In [59]:
service_model.is_service()

True

## Products

### Get Product Items

In [60]:
products_qs = entity_model.get_items_products()
pd.DataFrame(products_qs.values())

,created,updated,uuid,name,item_id,item_number,item_role,item_type,uom_id,sku,...,is_product_or_service,sold_as_unit,inventory_account_id,inventory_received,inventory_received_value,cogs_account_id,earnings_account_id,expense_account_id,additional_info,entity_id
0,2023-11-13 17:38:44.771410+00:00,2023-11-13 17:38:44.771427+00:00,7ae17dab-05c5-45da-9482-d670efde7db4,Product #1805,NUpQGqHNWCVRWQGeEbUK,IPR-0000000001,product,None,924d4b04-28a5-4ae9-9eba-fae474227567,L9LS56B6QJ6U,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
1,2023-11-13 17:38:44.771469+00:00,2023-11-13 17:38:44.771473+00:00,7ff6129d-6f61-4bd1-b9d4-3f930a727c92,Product #3815,ONMrNT8efwtEuRIjPnaG,IPR-0000000002,product,None,924d4b04-28a5-4ae9-9eba-fae474227567,NCVTL6EMSW79,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,4.070,754.62,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
2,2023-11-13 17:38:44.771495+00:00,2023-11-13 17:38:44.771499+00:00,b151ab95-a8af-49a8-9d33-30843583f4fa,Product #1590,puAhCLvhstbYVVappxXu,IPR-0000000003,product,None,220735b7-7c99-485e-ac8d-70144eceb6aa,9HBIFG779VV8,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,3.730,2240.24,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
3,2023-11-13 17:38:44.771519+00:00,2023-11-13 17:38:44.771523+00:00,3d63352a-88b2-4a46-9dc4-5e8f081a95c2,Product #9081,a2Y7GHB2e9u0aOXtmXor,IPR-0000000004,product,None,0b4fd4cb-3573-481a-a746-9e0a769bfce8,BRIEF0VLCGW2,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,19.580,7023.82,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
4,2023-11-13 17:38:44.771542+00:00,2023-11-13 17:38:44.771546+00:00,d660561d-64e0-4de6-8a67-d1750b55fec5,Product #4155,G1NtoqLldLnVilwRhE6z,IPR-0000000005,product,None,01851ed2-a516-43df-9f8e-02e0cd93b11d,T75M8WK7ODWQ,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,5.820,3401.61,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
5,2023-11-13 17:38:44.771563+00:00,2023-11-13 17:38:44.771567+00:00,b5ecc7b6-a4c4-45cb-a961-1614c330ac55,Product #8910,yvjDhan2DdR0r9DPWVFZ,IPR-0000000006,product,None,924d4b04-28a5-4ae9-9eba-fae474227567,HTWFMOGFZLVS,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
6,2023-11-13 17:38:44.771585+00:00,2023-11-13 17:38:44.771589+00:00,225728c0-ab97-4c23-b308-6e35bdb54ce9,Product #1326,Q9L88mdkXeBUjVLgSoB5,IPR-0000000007,product,None,01851ed2-a516-43df-9f8e-02e0cd93b11d,6PDP4NJODU76,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
7,2023-11-13 17:38:44.771606+00:00,2023-11-13 17:38:44.771610+00:00,be09b831-8bc3-429b-879d-fd003cb6449b,Product #9838,C3XiwEoBOcPxkJ0cpmlv,IPR-0000000008,product,None,924d4b04-28a5-4ae9-9eba-fae474227567,167ELR3237TC,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,26.080,5589.98,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
8,2023-11-13 17:38:44.771627+00:00,2023-11-13 17:38:44.771631+00:00,e7101625-3667-4945-953b-0bd99c34702c,Product #8274,vmdtW4EgC9abB87rDVly,IPR-0000000009,product,None,924d4b04-28a5-4ae9-9eba-fae474227567,G5RHL1EK2V3C,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
9,2023-11-13 17:38:44.771648+00:00,2023-11-13 17:38:44.771652+00:00,c00a81d0-62cf-4a5f-b5bf-e3759777e028,Product #5818,88

### Create Product Items

In [61]:
product_model = entity_model.create_item_product(
    name='1/2" Premium PVC Pipe',
    uom_model=uom_model_ft,
    item_type=ItemModel.ITEM_TYPE_MATERIAL
)

In [62]:
product_model.is_product()

True

## Inventory

### Get Inventory Items

In [63]:
inventory_qs = entity_model.get_items_inventory()
pd.DataFrame(inventory_qs.values())

,created,updated,uuid,name,item_id,item_number,item_role,item_type,uom_id,sku,...,is_product_or_service,sold_as_unit,inventory_account_id,inventory_received,inventory_received_value,cogs_account_id,earnings_account_id,expense_account_id,additional_info,entity_id
0,2023-11-13 17:38:44.771410+00:00,2023-11-13 17:38:44.771427+00:00,7ae17dab-05c5-45da-9482-d670efde7db4,Product #1805,NUpQGqHNWCVRWQGeEbUK,IPR-0000000001,product,None,924d4b04-28a5-4ae9-9eba-fae474227567,L9LS56B6QJ6U,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
1,2023-11-13 17:38:44.771469+00:00,2023-11-13 17:38:44.771473+00:00,7ff6129d-6f61-4bd1-b9d4-3f930a727c92,Product #3815,ONMrNT8efwtEuRIjPnaG,IPR-0000000002,product,None,924d4b04-28a5-4ae9-9eba-fae474227567,NCVTL6EMSW79,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,4.070,754.62,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
2,2023-11-13 17:38:44.771495+00:00,2023-11-13 17:38:44.771499+00:00,b151ab95-a8af-49a8-9d33-30843583f4fa,Product #1590,puAhCLvhstbYVVappxXu,IPR-0000000003,product,None,220735b7-7c99-485e-ac8d-70144eceb6aa,9HBIFG779VV8,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,3.730,2240.24,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
3,2023-11-13 17:38:44.771519+00:00,2023-11-13 17:38:44.771523+00:00,3d63352a-88b2-4a46-9dc4-5e8f081a95c2,Product #9081,a2Y7GHB2e9u0aOXtmXor,IPR-0000000004,product,None,0b4fd4cb-3573-481a-a746-9e0a769bfce8,BRIEF0VLCGW2,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,19.580,7023.82,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
4,2023-11-13 17:38:44.771542+00:00,2023-11-13 17:38:44.771546+00:00,d660561d-64e0-4de6-8a67-d1750b55fec5,Product #4155,G1NtoqLldLnVilwRhE6z,IPR-0000000005,product,None,01851ed2-a516-43df-9f8e-02e0cd93b11d,T75M8WK7ODWQ,...,True,False,12ed5dee-51dc-4d83-8354-5434fc19da18,5.820,3401.61,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,05321820-8593-4123-a93f-b4f278778ca5,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,2023-11-13 17:38:44.902353+00:00,2023-11-13 17:38:44.902356+00:00,3921a1f8-2b06-4beb-9920-cd9955f5a6aa,Inventory 2342,jbz4qDsYOGsTmQdHYdZ9,INV-0000000096,inventory,E,0b4fd4cb-3573-481a-a746-9e0a769bfce8,SSZ27T8P4HM7,...,False,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,None,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
58,2023-11-13 17:38:44.902371+00:00,2023-11-13 17:38:44.902374+00:00,0b294f30-9ee7-4c6c-9159-6e0a17e4c47f,Inventory 6793,iKkt1Pl2hd2bNDQhJINc,INV-0000000097,inventory,M,01851ed2-a516-43df-9f8e-02e0cd93b11d,L8V0H404B1KH,...,False,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,None,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
59,2023-11-13 17:38:44.902389+00:00,2023-11-13 17:38:44.902392+00:00,e073d34f-9426-45ad-ad37-f8dd8d76bb2d,Inventory 2774,VroOz5TzMboQDF4quCv3,INV-0000000098,inventory,L,0b4fd4cb-3573-481a-a746-9e0a769bfce8,9OQ08A8M33IX,...,False,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,None,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c
60,2023-11-13 17:38:44.902407+00:00,2023-11-13 17:38:44.902410+00:00,924a0e45-29ab-4d78-b202-9eb5864c4f75,Inventory 3015,uCcn3Iw96d3lpugd69hZ,INV-0000000099,inventory,S,94cf9411-f494-4d01-8c20-2f57b5e5fd6c,N6ILD9O4OMVE,...,False,False,12ed5dee-51dc-4d83-8354-5434fc19da18,0.000,0.00,21bf14d1-9c99-4323-9b41-cdfe1a0b4548,None,None,{},44d57e0b-48bc-4e1b-8d9f-04b781cc607c


### Create Inventory Items

In [64]:
inventory_model = entity_model.create_item_inventory(
    name='A Home to Flip!',
    uom_model=uom_model_unit,
    item_type=ItemModel.ITEM_TYPE_LUMP_SUM
)

In [65]:
inventory_model.is_inventory()

True

# Financial Statements

## Balance Sheet

In [66]:
bs_report = entity_model.get_balance_sheet_statement(
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
# save_pdf=True saves the PDF report in the project's BASE_DIR.
# filename and filepath may also be specified...
# will raise not implemented error if PDF support is not enabled...

### Balance Sheet Statement Raw Data

In [67]:
bs_report.get_report_data()

{'assets': {'total_balance': Decimal('129650.46000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('690baf2c-e77f-4f0e-b4fb-46eb02696410'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('132732.4000000000'),
      'balance_abs': Decimal('132732.4000000000')}],
    'total_balance': Decimal('132732.4000000000'),
    'role_name': 'Current Asset'},
   'asset_ca_recv': {'accounts': [{'account_uuid': UUID('f6de4cef-abfa-416d-b115-42eab4290a4b'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': None,
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_recv',
      'code': '1100',
      'name': 'Accounts Receivable',


## Income Statement

In [68]:
ic_report = entity_model.get_income_statement(
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
# save_pdf=True saves the PDF report in the project's BASE_DIR.
# filename and filepath may also be specified...
# will raise not implemented error if PDF support is not enabled...

### Income Statement Raw Data

In [69]:
ic_report.get_report_data()

{'operating': {'revenues': [{'account_uuid': UUID('05321820-8593-4123-a93f-b4f278778ca5'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('45627.3100000000'),
    'balance_abs': Decimal('45627.3100000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('21bf14d1-9c99-4323-9b41-cdfe1a0b4548'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-11403.8500000000'),
    'balance_abs': Decimal('11403.8500000000'),
    'role_name': 'Cost of Goods Sold'}],
  'expenses': [{'ac

## Cash Flow Statement

In [70]:
cf_report = entity_model.get_cash_flow_statement(
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
# save_pdf=True saves the PDF report in the project's BASE_DIR.
# filename and filepath may also be specified...

### Cash Flow Statement Raw Data

In [71]:
cf_report.get_report_data()

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('28320.3800000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': Decimal('-1593.8700000000')},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('4675.81000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': Decimal('1330.080000000000')},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('32732.4000000000000'),
  'FINANCING': Decimal('88698.

## All Financial Statements Data in a single Call

In [72]:
reports = entity_model.get_financial_statements(
    user_model=user_model,
    from_date=date(2022, 1, 1),
    to_date=date(2022, 12, 31),
    save_pdf=True,
    filepath='./'
)
# save_pdf=True saves the PDF report in the project's BASE_DIR.
# filename and filepath may also be specified...

In [73]:
reports.balance_sheet_statement.get_report_data()

{'assets': {'total_balance': Decimal('129650.46000000000'),
  'is_block': True,
  'roles': {'asset_ca_cash': {'accounts': [{'account_uuid': UUID('690baf2c-e77f-4f0e-b4fb-46eb02696410'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': 'fin_equity',
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('100000'),
      'balance_abs': Decimal('100000')},
     {'account_uuid': UUID('690baf2c-e77f-4f0e-b4fb-46eb02696410'),
      'unit_uuid': None,
      'unit_name': None,
      'activity': 'fin_std',
      'period_year': None,
      'period_month': None,
      'role_bs': 'assets',
      'role': 'asset_ca_cash',
      'code': '1010',
      'name': 'Cash',
      'balance_type': 'debit',
      'tx_type': None,
      'balance': Decimal('-11301.0400000000'),
      'balance_abs': Decimal('11301.040

In [74]:
reports.income_statement.get_report_data()

{'operating': {'revenues': [{'account_uuid': UUID('05321820-8593-4123-a93f-b4f278778ca5'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'in_operational',
    'code': '4010',
    'name': 'Sales Income',
    'balance_type': 'credit',
    'tx_type': None,
    'balance': Decimal('45627.3100000000'),
    'balance_abs': Decimal('45627.3100000000'),
    'role_name': 'Operational Income'}],
  'cogs': [{'account_uuid': UUID('21bf14d1-9c99-4323-9b41-cdfe1a0b4548'),
    'unit_uuid': None,
    'unit_name': None,
    'activity': None,
    'period_year': None,
    'period_month': None,
    'role_bs': 'equity',
    'role': 'cogs_regular',
    'code': '5010',
    'name': 'Cost of Goods Sold',
    'balance_type': 'debit',
    'tx_type': None,
    'balance': Decimal('-11403.8500000000'),
    'balance_abs': Decimal('11403.8500000000'),
    'role_name': 'Cost of Goods Sold'}],
  'expenses': [{'ac

In [75]:
reports.cash_flow_statement.get_report_data()

{'operating': {'GROUP_CFS_NET_INCOME': {'description': 'Net Income',
   'balance': Decimal('28320.3800000000000')},
  'GROUP_CFS_OP_DEPRECIATION_AMORTIZATION': {'description': 'Depreciation & Amortization of Assets',
   'balance': 0},
  'GROUP_CFS_OP_INVESTMENT_GAINS': {'description': 'Gain/Loss Sale of Assets',
   'balance': 0},
  'GROUP_CFS_OP_ACCOUNTS_RECEIVABLE': {'description': 'Accounts Receivable',
   'balance': Decimal('-1593.8700000000')},
  'GROUP_CFS_OP_INVENTORY': {'description': 'Inventories',
   'balance': Decimal('4675.81000000000')},
  'GROUP_CFS_OP_ACCOUNTS_PAYABLE': {'description': 'Accounts Payable',
   'balance': Decimal('1330.080000000000')},
  'GROUP_CFS_OP_OTHER_CURRENT_ASSETS_ADJUSTMENT': {'description': 'Other Current Assets',
   'balance': 0},
  'GROUP_CFS_OP_OTHER_CURRENT_LIABILITIES_ADJUSTMENT': {'description': 'Other Current Liabilities',
   'balance': 0}},
 'net_cash_by_activity': {'OPERATING': Decimal('32732.4000000000000'),
  'FINANCING': Decimal('88698.